# Example Notebook
How to:
- Calculate Galactocentric Position Velocity from Observables
- Calculate Energy, Lz, and Lperp in the potential of Dodd23
- Identify the grouping of the star from its position in IoM space

## Load Test Data
Test data is a small subset of the GaiaDR3 Halo dataset of Dodd23.

In [1]:
import id_Dodd23

In [2]:
rtsh

NameError: name 'rtsh' is not defined

In [ ]:
import astropy
import numpy as np
from astropy.table import QTable
from id_Dodd23.coordinates import add_units

In [ ]:
from id_Dodd23.load_data import test_data_file

In [ ]:
astro_tab = QTable.read(test_data_file)
astro_tab = add_units(astro_tab)

In [ ]:
astro_tab

## Coordinate Transform
Observables to Galacto_PosVel

In [ ]:
from id_Dodd23.coordinates import coord_transform_icrs_Galacto

In [ ]:
observables = ["ra","dec","distance","pmra","pmdec","radial_velocity"]
obs_data = {key:astro_tab[key]for key in observables}

In [ ]:
xyz = coord_transform_icrs_Galacto(obs_data)

## Calculate Dynamics
From galacto PosVel to \[En,Lz,Lperp\]

In [ ]:
from id_Dodd23.dynamics import dynamics_calc_H99

In [ ]:
vec_ELzLp  = dynamics_calc_H99(xyz)

In [ ]:
print(vec_ELzLp[:5,:])

In [ ]:
print(astro_tab[:5][["E","Lz","Lp"]])

## Identify Groups
Using the position in IoM space, identify stars that are close to known groups

In [ ]:
from id_Dodd23.id_stars import  add_maha_members_to_groups

In [ ]:
from id_Dodd23.load_data import named_Groups, group_covar,group_mean 

In [ ]:
labels = add_maha_members_to_groups(vec_ELzLp,group_mean,group_covar)

In [ ]:
original_labels = np.asarray(astro_tab["derived_labels_group"]).astype(int)

In [ ]:
(original_labels==labels).all()

In [ ]:
groups = named_Groups[labels]
groups[labels==-1] = "Other"

In [ ]:
for g in named_Groups:
    print(g, (groups==g).sum())